<a href="https://colab.research.google.com/github/sumeetk8/groq-langchain-chatbot/blob/master/langgraph_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install langgraph langsmith
!pip install langchain langchain_groq langchain_community

In [ ]:
# Import userdata from Colab to retrieve API keys
from google.colab import userdata

groq_api_key = userdata.get('groq_api_key')  # Retrieve the Groq API key
langsmith_api_key = userdata.get('langsmith_api_key')  # Retrieve the LangSmith API key

In [ ]:
# Environment variable setup for LangSmith tracing and project identification
import os
os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "CourseLanggraph"

In [ ]:
# Import ChatGroq for language model interaction
from langchain_groq import ChatGroq

In [ ]:
# Initialize the ChatGroq LLM with the specified model and API key
llm = ChatGroq(groq_api_key=groq_api_key, model_name="gemma2-9b-it")

In [ ]:
# Import necessary types for StateGraph
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
# Define the state structure for the chatbot using TypedDict
class State(TypedDict):
    messages: Annotated[list, add_messages]  # Annotated to specify a list of messages

In [ ]:
# Create an instance of StateGraph for managing state transitions
graph_builder = StateGraph(State)

In [ ]:
# Define the chatbot function that invokes the LLM based on the current state
def chatbot(state: State):
    return {"messages": llm.invoke(state['messages'])}  # LLM processes the input messages

In [ ]:
# Add the chatbot function as a node to the state graph
graph_builder.add_node("chatbot", chatbot)

In [ ]:
# Add edges to the state graph defining the start and end points
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# Compile the state graph for execution
graph = graph_builder.compile()

In [ ]:
# Optional: Display the state graph as an image if possible
from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass  # If an error occurs, skip image display

In [ ]:
# Interactive loop for the chatbot to receive user input and generate responses
while True:
    user_input = input("User: ")  # Prompt the user for input
    if user_input.lower() in ["quit", "q"]:  # Check for quit command
        print("Good Bye")
        break

    # Stream responses from the graph based on the input message
    for event in graph.stream({'messages': [("user", user_input)]}):
        print("Event:", event)  # Print the raw event for debugging
        print("Event Values:", event.values())  # Print event values to inspect contents

        if event.values() is None:
            print("No values in event.")  # Handle cases where event returns None
        else:
            for value in event.values():
                if value is not None:
                    print(value['messages'])  # Print the full message content
                    print("Assistant:", value["messages"].content)  # Print the assistant's response
                else:
                    print("Received NoneType value")  # Handle NoneType values explicitly